In [1]:
!pip install nltk -q
!pip install pip3-autoremove -q
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124 -q
!pip install unsloth
!pip install --upgrade "transformers>=4.53.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 39.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━

In [3]:
import gradio as gr
import nltk
from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer
import torch
from unsloth import FastLanguageModel
from datetime import datetime

# Download NLTK data
nltk.download('punkt', quiet=True)

# Initialize sentence embedder
embedder = SentenceTransformer('all-MiniLM-L12-v2')

# Model configuration
max_seq_length = 4096
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
load_in_4bit = True

def get_prompt_templates():
    templates = {
        "cot": """
You are a legal assistant drafting a clause compliant with the Companies Act 2013 of India. Follow these steps:

1. Analyze: Understand the requirement: {user_input}.
2. Extract: Identify key phrases in the requirement to include verbatim or closely paraphrase.
3. Identify: Recall relevant sections of the Companies Act 2013.
4. Draft: Write a clear, professional clause in formal legal language, ensuring key phrases from the requirement are included.
5. Validate: Ensure the clause aligns semantically with the requirement and complies with applicable laws.

Instructions:
- Use formal legal language.
- Include key phrases from the requirement to ensure close alignment.
- Reference specific sections where applicable.
- Do not include placeholder text (e.g., Your Name).
- Format as:
 Clause title
 Clause text

Drafted Clause:
"""
    }
    return templates

def load_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    return model, tokenizer

def generate_clause(user_input, strategy, model, tokenizer):
    templates = get_prompt_templates()
    prompt_template = templates[strategy]
    prompt = prompt_template.format(user_input=user_input)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=1000,
        do_sample=False,
        temperature=0.5,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    clause_start = response.find("Drafted Clause:")
    if clause_start != -1:
        clause_start += len("Drafted Clause:\n")
        clause = response[clause_start:].strip()
    else:
        clause = response.strip()
    sentences = sent_tokenize(clause)
    if sentences and not sentences[-1].endswith(('.', '!', '?')):
        sentences = sentences[:-1]
    clause = ' '.join(sentences)
    return clause

def generate_preamble(contract_type):
    return f"""THIS {contract_type.upper()} (the "Agreement") is made and entered into on this {datetime.now().strftime("%d day of %B, %Y")}, by and between [Company Name], a company incorporated under the Companies Act 2013 of India, having its registered office at [Company Address] (hereinafter referred to as the "Company"), and [Employee name], an individual residing at [Employee Address] (hereinafter referred to as the "Employee") (collectively referred to as "Parties").

This Agreement shall be effective on the date of signing this Agreement (hereinafter referred to as the “Effective Date”), throughout the period of employment of the employee and will remain in effect for a period of [Time Period] after the conclusion of the employment contract.
"""

def generate_closing():
    return """This Agreement shall be governed by and construed in accordance with the laws of India, without regard to its conflict of laws principles.

Any disputes arising out of or in connection with this Agreement shall be resolved through arbitration in Mumbai, India, in accordance with the Arbitration and Conciliation Act, 1996.

This Agreement constitutes the entire understanding between the parties with respect to the subject matter hereof and supersedes all prior agreements, understandings, and arrangements, whether written or oral.

The Parties hereby agree to the terms and conditions set forth in this Agreement and such is demonstrated throughout by their signatures below:

___________________________
[Company Name]
By: [Authorized Signatory]
[Date]

___________________________
[Employee Name]
[Date]
"""

def generate_full_contract(contract_type, clause_requirements):
    model, tokenizer = load_model()
    preamble = generate_preamble(contract_type)
    closing = generate_closing()
    clauses = []
    strategy = "cot"
    
    for idx, requirement in enumerate(clause_requirements, 1):
        clause = generate_clause(requirement.strip(), strategy, model, tokenizer)
        clauses.append(f"{idx}. {clause}\n")
    
    contract_content = f"{contract_type.upper()}\n\n"
    contract_content += preamble + "\n"
    contract_content += "TERMS AND CONDITIONS\n\n"
    contract_content += "\n\n".join(clauses) + "\n\n"
    contract_content += closing
    
    return contract_content

def gradio_interface(contract_type, clause_requirements):
    if not contract_type or not clause_requirements:
        return "Please provide both contract type and clause requirements."
    
    clause_requirements_list = [req.strip() for req in clause_requirements.split(",")]
    contract_content = generate_full_contract(contract_type, clause_requirements_list)
    return contract_content

# Define Gradio interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Contract Type", placeholder="e.g., Employment Contract, Shareholder's Agreement"),
        gr.Textbox(label="Clause Requirements (comma-separated)", placeholder="e.g., Draft a clause for absence at workplace, Draft a clause for confidentiality", lines=3)
    ],
    outputs=gr.Textbox(label="Generated Contract"),
    title="Contract Generator",
    description="Enter the contract type and clause requirements to generate a legal contract compliant with the Companies Act 2013 of India."
)

# Launch the interface
if __name__ == "__main__":
    iface.launch()

2025-07-11 20:35:33.745377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752266133.936978      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752266133.994442      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_36/3314325191.py:6: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
